### 워크플로우 이니시

In [1]:
import uuid
from pathlib import Path
from typing import List, Dict
from datetime import datetime

from entity.process import GuardCondition, TaskSpec, TaskType, Layer, AgentNature, AgentRole, Process
from entity.validators import TokenValidator, SpecChainValidator
from entity.tokens import Token
from core.utils import load_resource_specs
from core.tokenDB import TokenRepository
import os
import logging
from core import logging_utils
from pathlib import Path
from core.tokenDB import TokenRepository
from core.utils import load_cfg

# 0. master config 불러오기
cfg = load_cfg(Path().parent.parent / "cfg" / "prod.yaml")
log_levels = {
    "DEBUG": logging.DEBUG,
    "INFO": logging.INFO,
    "WARNING": logging.WARNING,
    "ERROR": logging.ERROR,
    "CRITICAL": logging.CRITICAL
}

log_level = log_levels.get(os.environ.get("LOG_LEVEL", "INFO").upper(), logging.INFO)

logging_utils.setup_logging(
    log_dir="logs",
    log_level=log_level,
    max_bytes=10 * 1024 * 1024, 
    backup_count=5, 
    console_output=True 
)

logging_utils.cleanup_old_logs(log_dir="logs", days_to_keep=7)

16:26:49 - core.logging_utils - INFO - Logging initialized - logs directory: c:\Users\kakao\Desktop\AI_Agent\Semantic Layer\logs
16:26:49 - core.logging_utils - INFO - Log rotation configured: max 10.0MB per file, keeping 5 backups


### entity/prompts.py

In [9]:
from entity import prompts

### LLM Invoke 예시

In [2]:
from langchain.chat_models import init_chat_model

llm_client = init_chat_model(model = "gpt-3.5-turbo", model_provider="openai")
print(getattr(llm_client.invoke("hey jude"), 'content'))

16:26:58 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Don't make it bad 
Take a sad song and make it better 
Remember to let her into your heart 
Then you can start to make it better 

Hey Jude, don't be afraid 
You were made to go out and get her 
The minute you let her under your skin 
Then you begin to make it better 

And anytime you feel the pain, hey Jude


### DB 예시

In [3]:
# 0. master config 불러오기
cfg = load_cfg(Path().parent.parent / "cfg" / "prod.yaml")

# 1. repo 불러오기
repo = TokenRepository(cfg['location']['token_db'], table_name="SOHO_LOAN_raw_knowledge")

In [4]:
repo.load(table_name_load="SOHO_LOAN_raw_knowledge", trace_id = "TKN_금융연수원_소호 여신심사_0000")


[Token: TKN_금융연수원_소호 여신심사_0000]
 ├─ Timestamp: 2026-02-03 14:24:23.807
 └─ Content  : # 소호
# 여신심사
SOHO CREDIT MANAGEMENT

백운수 저

![표지 그림: 건물 위에서 사람들이 동전(₩)을 옮기거나 망원경을 보는 일러스트]

**한국금융연수원**
KOREA BANKING INSTITUTE



---

(빈 페이지)



---

# 머리말

필자가 한국금융연수원에서 수년간 기업신용분석과 여신심사사례를 강의해 오면서 여러 수강자님으로부터 건의를 받은 내용이 바로 소기업과 소상공인에 관한 평가방법을 소개하여 달라는 주문이었습니다. 대기업들은 대부분 상장기업으로서 외부 전문신용평가회사의 신용등급이 공시되고 있을 뿐만 아니라 그들은 필요한 자금수요를 주로 회사채 발행이나 유상증자 등 직접금융에 의존하고 있는 추세이기 때문에 국내 금융기관들은 여신의 운용이 더욱 어려워져 중소기업이나 소호 대출 또는 순수 개인들에 대한 모기지론에 집중하고 있는 현실을 볼 때 수강자님들의 이와 같은 건의가 의미 있는 주문이라고 생각하였습니다.

이러한 요청과 국내 여신시...

In [10]:
import json
import logging
from typing import Dict, Any, List, Optional
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import HumanMessage

# (가정) 앞서 만든 TokenRepository, Token 클래스 import
# from db_manager import TokenRepository
# from token_schema import Token

class Agent:
    def __init__(self, llm, db_repo):
        """
        self.llm = llm 인스턴스
        self.repo = Agent의 지식 베이스
        """
        self.llm = llm
        self.repo = db_repo
        self.logger = logging.getLogger(__name__)
        self.logger.info("TokenAnalysisAgent initialized.")

    def run(self, state: Dict[str, Any], config: RunnableConfig) -> Dict[str, Any]:
        """
        [Logic]
        1. State에서 타겟 토큰 추출
        2. DB에서 연관된 Reference Token 조회 (Context)
        3. LLM Invoke (Prompt = Context + Target)
        4. 결과 파싱 및 State 업데이트
        """
        
        # 1. Extract Target Token from State
        # (State에 현재 처리 중인 토큰이 'current_token' 키로 있다고 가정)
        current_token_data = state.get("current_token")
        if not current_token_data:
            self.logger.warning("No token found in state to analyze.")
            return {}

        # Pydantic 모델로 변환 (편의상 딕셔너리 그대로 사용도 가능하나, 메서드 활용 위해 변환 권장)
        # target_token = Token(**current_token_data) 
        target_token = current_token_data # 여기선 Dict로 가정하고 진행

        # 2. agent role / type 추출
        agent_role = state.get("agent_role", None)
        agent_nature = state.get("agent_nature", None)
        
        # 2. [Routing] AgentRole 별 프롬프트 결정
        # LLM 타입인 경우에만 prompts.py의 베이스 프롬프트 주입
        final_system_prompt = task_config.get("system_prompt", "You are an analyst.")
        
        if agent_nature == "LLM":
            role_base = self._get_role_base_prompt(agent_role)
            # 베이스 페르소나 + 태스크 고유 지침 결합
            final_system_prompt = f"{role_base}\n\n[Task Instruction]\n{final_system_prompt}"
        
        # Task Config 추출 (프롬프트, 토픽 설정 등)
        task_config = state.get("task_config", {})        
        retrieval_topics = task_config.get("retrieval_topics", [])

        # 2. Context Retrieval (Single DB Logic)
        # 설정된 토픽이 없으면 타겟 토큰의 토픽을 그대로 사용
        search_keys = retrieval_topics if retrieval_topics else list(target_token.get("topics", {}).keys())
        
        try:
            # repo.find_context_tokens는 List[Token] 객체를 반환한다고 가정
            context_tokens = self.repo.find_context_tokens(search_keys, limit=5)
        except Exception as e:
            self.logger.error(f"Context retrieval failed: {e}")
            context_tokens = []

        # 3. Build Prompt
        context_str = self._format_context(context_tokens)
        
        full_prompt = f"""
        {final_system_prompt}

        [Reference Knowledge (Context)]
        The following are related tokens retrieved from the internal database:
        {context_str if context_tokens else "No related context found."}

        [Target Input (Token to Analyze)]
        - Content: {json.dumps(target_token.get('content', {}), ensure_ascii=False)}
        - Metadata: {target_token.get('topics', {})}

        [Instruction]
        Analyze the 'Target Input' based on the 'Reference Knowledge'.
        Check for consistency, factual accuracy, and compliance.
        
        Output must be a valid JSON object with the following structure:
        {{
            "analysis_summary": "...",
            "risk_score": 0.0 ~ 1.0,
            "is_compliant": true/false,
            "reasoning": "..."
        }}
        """

        try:
            # 4. Invoke LLM
            response = self.llm.invoke(full_prompt)
            response_content = response.content if hasattr(response, 'content') else str(response)

            # 5. Parse JSON
            analysis_data = json.loads(response_content.strip())

            # 6. Update Token Content (Evolution)
            # 기존 콘텐츠에 분석 결과를 병합
            updated_content = target_token.get('content', {}).copy()
            updated_content.update({
                "analysis_result": analysis_data,
                "referenced_ids": [t.trace_id for t in context_tokens] # Lineage 추적용
            })

            # 7. Return Updated State
            # State의 current_token을 업데이트된 내용으로 교체
            updated_token = target_token.copy()
            updated_token['content'] = updated_content
            
            # (로그 출력)
            if analysis_data.get("is_compliant"):
                self.logger.info(f"✅ Token {target_token.get('trace_id')} Analysis Passed.")
            else:
                self.logger.warning(f"⚠️ Token {target_token.get('trace_id')} Flagged: {analysis_data.get('reasoning')}")

            return {
                "current_token": updated_token,
                "analysis_result": analysis_data # 필요시 별도 키로도 반환
            }

        except json.JSONDecodeError as e:
            self.logger.error(f"Failed to parse analysis JSON: {e}")
            return {"error": "JSON Parsing Failed"}
            
        except Exception as e:
            self.logger.error(f"Token analysis execution failed: {e}")
            return {"error": str(e)}

    def _format_context(self, tokens: List) -> str:
        """Helper to format reference tokens into string"""
        formatted = ""
        for t in tokens:
            # t가 객체라면 t.content, 딕셔너리라면 t['content'] 처리 필요
            # 여기선 객체(Pydantic)라고 가정
            t_id = getattr(t, 'trace_id', 'Unknown')
            t_content = getattr(t, 'content', {})
            t_topics = getattr(t, 'topics', {})
            
            formatted += f"""
            ---
            [Ref Token ID: {t_id}]
            Topics: {t_topics}
            Content Summary: {str(t_content)[:300]}...
            """
        return formatted
    
    def _get_role_base_prompt(self, role: str) -> str:
        """Role에 따른 프롬프트 매핑 (Internal Helper)"""
        mapping = {
            "SUPERVISOR": prompts.SUPERVISOR_PROMPT,
            "CONSULTANT": prompts.CONSULTANT_PROMPT,
            "WORKER": prompts.WORKER_PROMPT
        }
        return mapping.get(role, prompts.DEFAULT_PROMPT)

### entity/agents.py